# Baseline CAD1 Task2

This tutorial walks through the process of running the CAD1 Task2 baseline using the shell Interface



---

## Setting the Location of the Project

For convenience, we are setting an environment variable with the location of the root working directory of the project. This variable will be used in various places throughout the tutorial. Please change this value to reflect where you have installed this notebook on your system.

In [ ]:
import os
os.environ["NBOOKROOT"] = os.getcwd()
os.getenv("NBOOKROOT")

## Cloning the Clarity Repository
We first need to install the Clarity package.

In [2]:
print("Cloning git repo...")
!git clone --quiet https://github.com/claritychallenge/clarity.git
%cd clarity
%pip install -e .
import os
import sys

sys.path.append(f'{os.getenv("NBOOKROOT")}/clarity')
from IPython.display import clear_output

clear_output()
print("Repository installed")

Repository installed


## Get the demo data

We will be using music audio and listener metadata.

In [7]:
!gdown 1OWn0w39t_aeOOh514wRUgTw5uSwsoB17
!mv cadenza_task2_data_demo.tar.xz recipes/cad1/task2/baseline
!cd recipes/cad1/task2/baseline
!tar -xvf cadenza_task2_data_demo.tar.xz

clear_output()
print("Data installed")

Data installed


## Changing working Directory

Next, we change working directory to the location of the shell scripts we wish to run.

In [4]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad1/task2/baseline

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task2/baseline


## Inspecting Existing Configuration

Let's inspect the config.yml file.

```yaml
path:
  root: ../../cadenza_task2_data_demo/cad1/task2
  audio_dir: ${path.root}/audio
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.audio_dir}/music
  hrtf_dir: ${path.audio_dir}/eBrird
  listeners_file: ${path.metadata_dir}/listeners.valid.json
  scenes_file: ${path.metadata_dir}/scenes.json
  scenes_listeners_file: ${path.metadata_dir}/scenes_listeners.json
  hrtf_file: ${path.metadata_dir}/eBrird_BRIR.json
  exp_folder: ./exp # folder to store enhanced signals and final results

team_id: T001

sample_rate: 44100          # sample rate of the input signal
enhanced_sample_rate: 32000 # sample rate for the enhanced output signal

nalr:
  nfir: 220
  sample_rate: ${sample_rate}

compressor:
  threshold: 0.7
  attenuation: 0.1
  attack: 5
  release: 20
  rms_buffer_size: 0.064

soft_clip: False

enhance:
  average_level: -14   # Average level according Spotify's levels
  min_level: -19

evaluate:
  set_random_seed: True
  small_test: False
  save_intermediate_wavs: False
  split: valid # train, valid
  batch_size: 1   # Number of batches
  batch: 0      # Batch number to evaluate

# hydra config
hydra:
  run:
    dir: ${path.exp_folder}
```

## Running Enhancement

We are now ready to run the enhance.py python script. However, the standard configuration is designed to work with the full dataset. We can redirect the script to the correct folders by overriding the appropriate configuration parameters.

In [8]:
!python enhance.py path.root={os.environ['NBOOKROOT']}/clarity/recipes/cad1/task2/baseline/cad1/task2

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task2/baseline/evaluate.py:139: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task2/baseline/enhance.py:95: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/anaconda3/envs/clarity39/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
100%|█████████████████████

Now we have the enhanced output. Below, we can load and play the audio to listen to examples of the results.

In [53]:
from pathlib import Path
import IPython.display as ipd
from clarity.utils.flac_encoder import read_flac_signal

audio_path = Path("exp/enhanced_signals")
audio_files = [f for f in audio_path.glob('*/*/*') if f.suffix == '.flac'][:4]

for file_to_play in audio_files:
  signal, sample_rate = read_flac_signal(file_to_play)
  print(file_to_play.name)
  ipd.display(ipd.Audio(signal.T, rate=sample_rate))

FileNotFoundError: [Errno 2] No such file or directory

Now that we have enhanced audios we can use the evaluate recipe to generate HAAQI scores for the signals. The evaluation is run in the same manner as the enhancement script.

In [14]:
!python evaluate.py path.root={os.environ['NBOOKROOT']}/clarity/recipes/cad1/task2/baseline/cad1/task2

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task2/baseline/evaluate.py:139: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/anaconda3/envs/clarity39/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
[2024-08-21 10:57:38,292][__main__][INFO] - Evaluating from enhanced_signals directory
 10%|████▎                                      | 2/20 [04:23<39:09, 130.51s/it][2024-08-21 11:03:16,806][recipes.cad1.task2.baseline.car_scene_acoustics][WARNING] - Scene T041020_L5000_S2023: 24638 samples clipped in evaluation signal.
[2024-08-21 11:03:21,229]

In [ ]:
from pathlib import Path
from scipy.io import wavfile

import IPython.display as ipd

audio_path = Path("exp/evaluation_signals")
audio_files = [f for f in audio_path.glob('*/*/ha_*') if f.suffix == '.wav'][:1]

signals = []
rates = []
columns = []
for file_to_play in audio_files:
  sample_rate, signal = wavfile.read(file_to_play)
  print(file_to_play.parts[-3], file_to_play.parts[-2], file_to_play.name)
  ipd.display(ipd.Audio(signal.T, rate=sample_rate))

  print(file_to_play.parts[-3], file_to_play.parts[-2], "ref_signal_for_eval.wav")
  _, signal = wavfile.read(file_to_play.parent / "ref_signal_for_eval.wav")
  ipd.display(ipd.Audio(signal.T, rate=sample_rate))

We hope that this tutorial has been useful and has explained the process for using the recipe scripts using the Hydra configuration system. This approach can be applied to all of the recipes that are included in the repository.